In [3]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr
import math as math
from scipy.optimize import curve_fit
import seaborn as sns
import random as random


from netCDF4 import Dataset
import h5py

plt.style.use('seaborn')

C:\Users\pkoll\AppData\Local\Temp\ipykernel_13088\312763924.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [4]:
pip install apexpy

  Using cached apexpy-2.0.1.tar.gz (328 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [21 lines of output]
  + meson setup C:\Users\pkoll\AppData\Local\Temp\pip-install-_031fmr3\apexpy_c9d834603d4c46a88d083c7cffaee7ff C:\Users\pkoll\AppData\Local\Temp\pip-install-_031fmr3\apexpy_c9d834603d4c46a88d083c7cffaee7ff\.mesonpy-e_c6jk8c\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\pkoll\AppData\Local\Temp\pip-install-_031fmr3\apexpy_c9d834603d4c46a88d083c7cffaee7ff\.mesonpy-e_c6jk8c\build\meson-python-native-file.ini
  The Meson build system
  Version: 1.2.1
  Source dir: C:\Users\pkoll\AppData\Local\Temp\pip-install-_031fmr3\apexpy_c9d834603d4c46a88d083c7cffaee7ff
  Build dir: C:\Users\pkoll\AppData\Local\Temp\pip-install-_031fmr3\apexpy_c9d834603d4c46a88d083c7cffaee7ff\.mesonpy-e_c6jk8c\build
  Build type: native build
  Project name: apexpy
  Project version: 2.0.1
  
  ..\..\meson.build:1:0: ERROR: Unknown compil


  Using cached apexpy-2.0.1.tar.gz (328 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [21 lines of output]
  + meson setup C:\Users\pkoll\AppData\Local\Temp\pip-install-p5th1lgi\apexpy_a5328f7af5b4485cb0c0c71985ef3618 C:\Users\pkoll\AppData\Local\Temp\pip-install-p5th1lgi\apexpy_a5328f7af5b4485cb0c0c71985ef3618\.mesonpy-314i4arn\build -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\pkoll\AppData\Local\Temp\pip-install-p5th1lgi\apexpy_a5328f7af5b4485cb0c0c71985ef3618\.mesonpy-314i4arn\build\meson-python-native-file.ini
  The Meson build system
  Version: 1.2.1
  Source dir: C:\Users\pkoll\AppData\Local\Temp\pip-install-p5th1lgi\apexpy_a5328f7af5b4485cb0c0c71985ef3618
  Build dir: C:\Users\pkoll\AppData\Local\Temp\pip-install-p5th1lgi\apexpy_a5328f7af5b4485cb0c0c71985ef3618\.mesonpy-314i4arn\build
  Build type: native build
  Project name: apexpy
  Project version: 2.0.1
  
  ..\..\meson.build:1:0: ERROR: Unknown compil

In [2]:
dsm  = xr.open_dataset('eqcross_mv05iv06_r00_mti_2021 (1).nc')
dsi  = xr.open_dataset('eqcross_mv05iv06_r00_ivm_2021.nc')

dsi = dsi.load()

C:\Users\pkoll\anaconda3\lib\site-packages\ecmwflibs\__init__.py:83: UserWarning: DLL load failed while importing _ecmwflibs: The specified procedure could not be found.
  warnings.warn(str(e))
C:\Users\pkoll\anaconda3\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
dm0 = dsm[['u','v', 'q']].sel(dt=0).isel(alt=slice(2,-2)) # Only u, v variables, but also quality (q) variable for possible later filtering
di0 = dsi[['drift_mer','slt', 'time', 'dm_flag', 'rpa_flag']].sel(dt=0) # only use drift_mer variable (but also save other variables for context: local time, actual time, IVM data quality flags)

d = xr.merge((dm0, di0))
d = d.dropna(dim='num', how='any', subset=['u','drift_mer']) # Drop missing data

# Only keep later LTs and data where the ivm quality flag is good
d = d.where((d.slt > 11) & (d.slt < 16) & (d.q.mean(dim='alt') > 0.95) & (d.dm_flag == 0) & (d.rpa_flag == 0), drop=True)


In [ ]:
apex = apexpy.Apex(date=2021, refh=80.)

def add_mag_winds(ds, apex):
    '''
    Convert winds to modified apex coordinates.
    Also add basis vectors and coordinates:
        - Richmond d,e vectors
        - Mod apex coordinates for the wind profile, for sanity checks
   
    ds = xarray.Dataset for the north or south footpoint.
    apex = apexpy object
    '''
    # Careful with nans
    lat = ds.lat.values
    lon = ds.lon.values
    alt = ds.alt.values
    N = len(alt)
    d1 = np.nan * np.zeros((3,N))
    d2 = np.nan * np.zeros((3,N))
    d3 = np.nan * np.zeros((3,N))
    e1 = np.nan * np.zeros((3,N))
    e2 = np.nan * np.zeros((3,N))
    e3 = np.nan * np.zeros((3,N))
   
    # 2023 May 17 generalize to allow for either scalar lat/lon or altitude dependent
    if np.ndim(lat) == 0:
        lat = lat*np.ones_like(alt)
        lon = lon*np.ones_like(alt)
       
    idx = np.isfinite(lat) & np.isfinite(lon)
    if idx.sum() > 0: # If everything's nan, just let it stay nan
        _, _, _, _, _, _, d1i, d2i, d3i, e1i, e2i, e3i = apex.basevectors_apex(lat[idx], lon[idx], height=alt[idx], coords='geo')
        d1[:,idx] = d1i
        d2[:,idx] = d2i
        d3[:,idx] = d3i
        e1[:,idx] = e1i
        e2[:,idx] = e2i
        e3[:,idx] = e3i
   
    ds['d1'] = (('alt','vec_enu'), d1.T)
    ds['d2'] = (('alt','vec_enu'), d2.T)
    ds['d3'] = (('alt','vec_enu'), d3.T)
    ds['e1'] = (('alt','vec_enu'), e1.T)
    ds['e2'] = (('alt','vec_enu'), e2.T)
    ds['e3'] = (('alt','vec_enu'), e3.T)
   
    u = xr.concat((ds['u'], ds['v'], xr.zeros_like(ds['u'])), dim='vec_enu') # DataArray, wind vector, assuming zero vertical wind
   
    ds['ue1'] = (ds.d1 * u).sum(dim='vec_enu', min_count=3) # Dot product of wind vector and d1 vector, using enu coordinates. Preserve nans
    ds['ue2'] = (ds.d2 * u).sum(dim='vec_enu', min_count=3)
    ds['ue3'] = (ds.d3 * u).sum(dim='vec_enu', min_count=3)